[View in Colaboratory](https://colab.research.google.com/github/SakshiPriya/text-generation-word-by-word-using-rnn/blob/master/wordbyword.ipynb)

In [0]:
import numpy as np
import nltk
from  nltk.corpus import gutenberg
from nltk import word_tokenize
import unicodedata


In [0]:
nltk.download('gutenberg')
nltk.download('punkt')

[nltk_data] Downloading package gutenberg to /content/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
data=gutenberg.raw('shakespeare-hamlet.txt')
data=unicodedata.normalize('NFKD', data).encode('ascii','ignore')
dat=word_tokenize(data)
text=nltk.Text(dat)
unique=list(set(dat))
datalen,vocabsize=len(data),len(unique)
chartoint={b:a for a,b in enumerate(unique) }
inttochar={a:b for a,b in enumerate(unique) }


In [0]:
neuronno=1000
lr=0.01
timestamp=30

In [0]:
w1=2*np.random.random((vocabsize,neuronno))-1
w2=2*np.random.random((neuronno,neuronno))-1
w3=2*np.random.random((neuronno,vocabsize))-1
b1=np.random.random((1,neuronno))
b2=np.random.random((1,vocabsize))

In [0]:
def travelintimestamp(inputs,target,ainitial):
  x,y,a,p={},{},{},{}
  a[-1]=np.copy(ainitial)   #size(1,neuron)
  loss=0
  for i in range(len(inputs)):
    x[i]=np.zeros((1,vocabsize))
    x[i][0,inputs[i]]=1
    a[i]=np.tanh(np.dot(x[i],w1)+np.dot(a[i-1],w2)+b1)
    y[i]=np.dot(a[i],w3)+b2
    p[i]=np.exp(y[i])/np.sum(np.exp(y[i]))
    loss+=(-np.log(p[i][0,target[i]]))
    
  dw1,dw2,dw3=np.zeros_like(w1),np.zeros_like(w2),np.zeros_like(w3)
  db1,db2=np.zeros_like(b1),np.zeros_like(b2)
  daprev=np.zeros_like(a[0])
       
  for i in reversed(range(len(inputs))):
      dy=np.copy(p[i])
      dy[0,target[i]]-=1
      dw3+=np.dot(a[i].T,dy)
      db2+=dy
      da=np.dot(dy,w3.T)+daprev
      dat=np.dot(da,(1-np.dot(a[i].T,a[i])))
      db1+=dat
      dw1+=np.dot(x[i].T,dat)
      dw2+=np.dot(a[i-1].T,dat)
      daprev=np.dot(dat,w2)
           
  return loss,dw1,dw2,dw3,db1,db2,a[len(inputs)-1]
  

In [0]:
def sampleoftext(a,randno,length):
  x=np.zeros((1,vocabsize))
  x[0,randno]=1
  result=[]
  for  i in range(length):
    a=np.tanh(np.dot(x,w1)+np.dot(a,w2)+b1)
    y=np.dot(a,w3)+b2
    p=np.exp(y)/np.sum(np.exp(y))
    
    no=np.random.choice(range(vocabsize),p=p.ravel())
    x=np.zeros((1,vocabsize))
    x[0,no]=1
    result.append(no)
  return result

In [0]:
n,p=0,0
sdw1=np.zeros_like(w1)
sdw2=np.zeros_like(w2)
sdw3=np.zeros_like(w3)
sb1=np.zeros_like(b1)
sb2=np.zeros_like(b2)
weightedloss=0
while True:
  if p+timestamp>datalen or n==0:
    ainitial=np.zeros((1,neuronno))
    p=0
  inputs= [chartoint[char] for char in text[p:p+timestamp]]
  target= [chartoint[char] for char in text[p+1:p+timestamp+1]]
  
  loss,dw1,dw2,dw3,db1,db2,ainitial=travelintimestamp(inputs,target,ainitial)
  weightedloss+=loss
  if n%100==0:
    print('loss',n,loss)
    indexli=sampleoftext(ainitial,42,200)
    print(' '.join([inttochar[index] for index in indexli]))
    print('--------------------------------------------------------------------------------------')
    
  for par, dpar,s in zip([w1, w2, w3, b1, b2], [dw1, dw2, dw3, db1, db2], [sdw1, sdw2, sdw3, sb1, sb2]):
    s=dpar*dpar
    par-=lr*(dpar/np.sqrt(s+1e-7))
    
  p+=timestamp
  n+=1
  
print(w1,w2,w3)

('loss', 0, 2059.825997425463)
Quilles temperately amaz or shent Weazell chide waste scape front Incontinencie easie Thunder wary Claudio Frailty vnwholsome obseruant Bedrid pins knife curles could now harrow distrust Reads Capitol indeauour out-breake dayes By thou'lt taints Collected none Heart holds wonder Gentrie sea-gowne Tragedie perturbed Wantonnesse shut Arrowes Treacherous crimes Starres Absent Orchard peace-parted Deuices Purse Compulsatiue giue shuffling folke primall Imagination betime bulke part wrote Nunnery Starre drown'd hoopes best saile Somnet those encountred gone Nephewes carries lying fashion Soales Cum Woodcocks limbes farewell Prisoner Saluation thirties Directly Abuses shift memory Battery demand Ardure Handsaw iangled Meet Such Lights first purg lippes preaching lend Saint Graces Chace tempt few therfore manner intreat Shippe suffer beautifed wisenesse ioyfully conueyance withdraw lockt wring Ambass Pardon Maiesties Obey Land Put kist Tutor fairely two answer c